# Signing and verifying a block header 

## 1. Creating Encryption Key pair

In [1]:
def generate_RSA(bits=2048):
    '''
    Generate an RSA keypair with an exponent of 65537 in PEM format
    param: bits The key length in bits
    Return private key and public key
    '''
    from Crypto.PublicKey import RSA 
    new_key = RSA.generate(bits, e=65537) 
    public_key = new_key.publickey().exportKey("PEM") 
    private_key = new_key.exportKey("PEM") 
    return private_key, public_key


In [2]:
from Crypto.PublicKey import RSA 

alice_priv, alice_pub = generate_RSA()
bob_priv, bob_pub = generate_RSA()

with open("resources/alice_pub.pem", "wb") as file:
    file.write(alice_pub)

with open("resources/alice_priv.pem", "wb") as file:
    file.write(alice_priv)

with open("resources/bob_pub.pem", "wb") as file:
    file.write(bob_pub)

with open("resources/bob_priv.pem", "wb") as file:
    file.write(bob_priv)

## 2. Signing a blocker header using private key

In [3]:
import json 
from Crypto.Hash import SHA

def hash_block(msg=""):
    # Hashes a list of dictionaries
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    return SHA.new(str.encode(msg))

In [4]:
from Crypto.PublicKey import RSA

# Load resources
with open("resources/alice_priv.pem", "rb") as file:
    alice_priv = RSA.importKey(file.read())
    
with open('resources/alice_pub.pem', "rb") as file:
    alice_pub = RSA.importKey(file.read())

with open("resources/bob_priv.pem", "rb") as file:
    bob_priv = RSA.importKey(file.read())
    
with open('resources/bob_pub.pem', "rb") as file:
    bob_pub = RSA.importKey(file.read())

In [5]:
from Crypto.Signature import PKCS1_v1_5

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(block_header)
signer = PKCS1_v1_5.new(bob_priv)
signature = signer.sign(hash_obj)

signature

b'fG\x0f\xc1}4\xeb\xbf\xe9`\x80F*&\x1b\x7fA\xc9[9/\xf4\x04o\x80CH\x9e\xac\xfd"\xaf\xe7\xd9\xd9t|X\xc4s\xd4RM\xc4\xa7a"\xee\x97q\xe1\xcdg\xee\x91\xe2\xb8\xcd\x89]W8\xbb0n\x91\xfa8{\xd5R\x87\xc4_\t\xbbR\x85\x01=he\xda\x17\x1d]\xed\xa7m\xc5\x9a\xb4\x86\xf3\xf6\xb5i\x17\xa4\x18bB\\/\xd4\xeb\xc6\x8c\xae\x9f)\x1c4E\x8f\xdd\x1f8\xadM\xbci\xd4xq\xf9\xd1af\xca\xa4N\xc8\xf3\xfe\x89\x13\x9c\x95\xff\xd5\x13\xb6\x0e\x0c\xdbC\xe5VU^\x89~\xb1\x85<JX\xf0\xf6\x87\xdd.\xe6\xdb\xbc\x92\n\xc5\xd91\x8b\xe6\x11\xbc.\x90s\xd3$\xc6\xb30\xe7\xa3\xf3\r\xc6N\x87V7>\xd9\xb2\xf96\xec\xac\xec\x95YE\x1b\x83\xcc\x07\x95G\xde\xf2v\xc4.\xff\xaaH\x80\x91\xd6B\xc6\x91\x18\x9fh\xac\xe1kk\xa9\x90\x93M=\xfb\xc4\xf1\x02\xe8\x1b\x1fC(5N:\xbe?]\xad\x08\xbamdh'

In [6]:
import pickle

with open("resources/signature.pkl", "wb") as file: 
    pickle.dump(signature, file)

## 3. Verifying a block header using public key

In [7]:
from Crypto.Signature import PKCS1_v1_5
import pickle

with open("resources/signature.pkl", "rb") as file: 
    signature = pickle.load(file)

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(block_header)

verifier = PKCS1_v1_5.new(bob_pub)

result = verifier.verify(hash_obj, signature)

result

True